# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path=os.path.join('output_data', 'city_data.csv')
city_data=pd.read_csv(path)


In [3]:
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Severo-Kuril'sk,50.6789,156.1250,52.70,74,1,9.17,RU,1631733119
1,Hobart,-42.8794,147.3294,57.06,66,20,1.01,AU,1631736682
2,Shakawe,-18.3667,21.8500,76.69,15,2,5.53,BW,1631766450
3,Mataura,-46.1927,168.8643,46.47,84,89,3.71,NZ,1631731579
4,Albany,42.6001,-73.9662,64.27,94,100,2.17,US,1631788623


In [4]:
city_data.shape

(574, 9)

In [5]:
city_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 574 entries, 0 to 573
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        574 non-null    object 
 1   Lat         574 non-null    float64
 2   Lng         574 non-null    float64
 3   Max Temp    574 non-null    float64
 4   Humidity    574 non-null    int64  
 5   Cloudiness  574 non-null    int64  
 6   Wind Speed  574 non-null    float64
 7   Country     570 non-null    object 
 8   Date        574 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 40.5+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [7]:
locations=city_data[['Lat', 'Lng']]
weight=city_data['Humidity']

In [36]:
layout={
        'width': '1000px',
        'height': '800px',
        'padding': '3px',
        'border': '1px solid black'
}
fig=gmaps.figure(center=(46.0, -5.0), zoom_level=2,
                 layout=layout, map_type="HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations,weights=weight,  
                                 dissipating=False,max_intensity=100,
                                 point_radius=4.0)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
city_data.isna().sum()

City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       4
Date          0
dtype: int64

In [10]:
city_data.dropna(inplace=True)
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Severo-Kuril'sk,50.6789,156.1250,52.70,74,1,9.17,RU,1631733119
1,Hobart,-42.8794,147.3294,57.06,66,20,1.01,AU,1631736682
2,Shakawe,-18.3667,21.8500,76.69,15,2,5.53,BW,1631766450
3,Mataura,-46.1927,168.8643,46.47,84,89,3.71,NZ,1631731579
4,Albany,42.6001,-73.9662,64.27,94,100,2.17,US,1631788623
...,...,...,...,...,...,...,...,...,...
569,Kembé,4.6228,21.8865,72.07,94,100,1.81,CF,1631766195
570,Machilipatnam,16.1667,81.1333,91.60,65,94,7.27,IN,1631751837
571,San Fernando del Valle de Catamarca,-28.4696,-65.7852,55.58,29,0,5.99,AR,1631787579
572,Maracanã,-0.7667,-47.4500,73.47,98,7,1.68,BR,1631782890


In [19]:
hotel_df=city_data[(city_data['Max Temp']>70) 
          &(city_data['Max Temp']<80)
          &(city_data['Wind Speed']<10)
          &(city_data['Cloudiness']==0)       ]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Ubari,26.5921,12.7805,73.85,28,0,6.89,LY,1631768097
23,Saint-Pierre,-21.3393,55.4781,78.44,63,0,5.75,RE,1631758417
31,Piranshahr,36.7010,45.1413,75.42,14,0,5.99,IR,1631760148
146,Waddān,29.1614,16.1390,72.77,53,0,6.06,LY,1631767249
173,Misratah,32.3754,15.0925,77.61,63,0,6.69,LY,1631767446
264,Sabha,27.0377,14.4283,74.95,30,0,8.48,LY,1631767694
265,Nālūt,30.3333,10.8500,74.52,28,0,8.75,LY,1631768500
268,Talā,30.6803,30.9429,79.32,50,0,8.43,EG,1631763670
270,Ürümqi,43.8010,87.6005,77.95,22,0,4.47,CN,1631749788
309,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631767107


In [20]:
hotel_df['Hotel Name']=''
hotel_df["Hotel Address"]=''

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_49293/3511615655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=''
/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_49293/3511615655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"]=''


In [21]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
11,Ubari,26.5921,12.7805,73.85,28,0,6.89,LY,1631768097,,
23,Saint-Pierre,-21.3393,55.4781,78.44,63,0,5.75,RE,1631758417,,
31,Piranshahr,36.7010,45.1413,75.42,14,0,5.99,IR,1631760148,,
146,Waddān,29.1614,16.1390,72.77,53,0,6.06,LY,1631767249,,
173,Misratah,32.3754,15.0925,77.61,63,0,6.69,LY,1631767446,,
264,Sabha,27.0377,14.4283,74.95,30,0,8.48,LY,1631767694,,
265,Nālūt,30.3333,10.8500,74.52,28,0,8.75,LY,1631768500,,
268,Talā,30.6803,30.9429,79.32,50,0,8.43,EG,1631763670,,
270,Ürümqi,43.8010,87.6005,77.95,22,0,4.47,CN,1631749788,,
309,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631767107,,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

In [23]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # make request and print url
    hotel_add = requests.get(base_url, params=params)

    # convert to json
    hotel_add = hotel_add.json()
    #print(json.dumps(hotel_add, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_add["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_add["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/sudippandey/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result... skipping.


In [24]:
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
11,Ubari,26.5921,12.7805,73.85,28,0,6.89,LY,1631768097,Awbari‎,Awbari‎
23,Saint-Pierre,-21.3393,55.4781,78.44,63,0,5.75,RE,1631758417,Saint Pierre,Saint Pierre
31,Piranshahr,36.7010,45.1413,75.42,14,0,5.99,IR,1631760148,Piranshahr,Piranshahr
146,Waddān,29.1614,16.1390,72.77,53,0,6.06,LY,1631767249,Waddan,Waddan
173,Misratah,32.3754,15.0925,77.61,63,0,6.69,LY,1631767446,Misrata,Misrata
264,Sabha,27.0377,14.4283,74.95,30,0,8.48,LY,1631767694,Sabhā,Sabhā
265,Nālūt,30.3333,10.8500,74.52,28,0,8.75,LY,1631768500,,
268,Talā,30.6803,30.9429,79.32,50,0,8.43,EG,1631763670,Tala,Tala
270,Ürümqi,43.8010,87.6005,77.95,22,0,4.47,CN,1631749788,Ürümqi,Ürümqi
309,Sirte,31.2089,16.5887,75.27,59,0,4.38,LY,1631767107,Sirte,Sirte


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map
layout={
        'width': '1000px',
        'height': '800px',
        'padding': '3px',
        'border': '1px solid black'
}
fig=gmaps.figure(center=(46.0, -5.0), zoom_level=2,
                 layout=layout, map_type="HYBRID")
heatmap_layer = gmaps.heatmap_layer(locations,weights=weight,  
                                 dissipating=False,max_intensity=100,
                                 point_radius=4.0)

markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(heatmap_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='3px', width='1000px'))